<img src="https://electronaut.info/wp-content/uploads/2017/05/Tubescape_jupyter2.jpg">

<h1><font color = b30000>TUBE DATA MATCHER for uTRACER FILES</font></h1><h5>Anode current for a sweep of grid voltages  when anode voltage = constant</h5>


# STEP 1 — Import and process tube data, build dataframe.

In [1]:
from electronaut_uTracer_Import import *
df.head()

Please enter the path to the folder containing the uTracer files: /Users/rrmc/github_rrMacKinnon/Electrona_uTracer/SampleTubeData_1_to_196

What type of tubes are these? 5749

— — — — — 

Data from 167 tubes of type 5749 was successfully read.
A dataframe containing all 167 tubes was successfully created.


,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,1,5749,325,99.278,65.383,44.717,34.082,26.669,20.931,16.492,12.992,10.291,8.23,6.669,5.495,4.626
1,10,5749,325,84.662,55.752,37.5,27.666,20.873,15.84,12.088,9.304,7.261,5.807,4.747,3.945,3.344
2,100,5749,325,76.506,50.319,33.695,25.027,19.052,14.603,11.283,8.775,6.954,5.655,4.765,4.118,3.619
3,102,5749,325,92.106,59.906,39.752,29.517,22.626,17.543,13.745,10.894,8.787,7.228,6.08,5.204,4.575
4,103,5749,325,69.639,45.449,30.772,22.843,17.624,13.782,10.864,8.615,6.916,5.657,4.754,4.092,3.619


In [2]:
len(df)

167

##### TODO:  INSERT PLOT OF ALL TUBES


# STEP 3: Show the statistics of the tube data set

In [3]:
# For some reason, .describe is requiring the .astype(float) in order to show the stats.
df_stats = df.astype(float).describe()
df_stats.loc[:, 'tube_type'] = df_stats.loc[:, 'tube_type'].astype(int)
df_stats.loc[:, 'tube_ID'] = df_stats.loc[:, 'tube_ID'].astype(int)
# df_stats.loc[:, 'tube_ID'] = df_stats.loc[:, 'tube_ID'].astype(str)
df_stats

,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
count,167,167,167.0,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000
mean,94,5749,325.0,85.854024,54.600138,35.936389,26.606778,20.321790,15.648988,12.173521,9.577371,7.657347,6.248994,5.208695,4.410192,3.798886
std,57,0,0.0,10.252616,5.708263,3.792364,2.876220,2.387349,2.053135,1.794487,1.576286,1.382130,1.206177,1.052825,0.929768,0.840940
min,1,5749,325.0,48.494000,30.962000,19.730000,14.837000,11.473000,8.866000,6.827000,5.260000,4.091000,3.235000,2.613000,2.153000,1.833000
25%,43,5749,325.0,80.143500,51.841500,34.041500,25.167500,19.052000,14.428000,11.082500,8.645000,6.855500,5.501500,4.584500,3.869500,3.319500
50%,94,5749,325.0,87.154000,55.362000,36.408000,26.777000,20.534000,15.759000,12.180000,9.602000,7.679000,6.162000,5.096000,4.343000,3.712000
75%,143,5749,325.0,91.902000,58.625500,38.192000,28.252000,21.554500,16.662500,13.070000,10.400000,8.427000,6.923000,5.801500,4.957500,4.302500
max,196,5749,325.0,107.892000,65.654000,46.996000,36.666000,30.080000,25.292000,21.460000,18.384000,15.898000,13.867000,12.198000,10.760000,9.616000


##### TODO:  INSERT PLOT(S) OF STATISTICS 

# STEP 4: Calculate the differences between all tubes

In [4]:
def build_difference_df(df, df_stats):
       
    # Make a list of all the columns in the dataframe
    cols = list(df.columns)
    
    # Make a list of just the bias columns
    bias_list = []
    for i in cols:
        if i.startswith('Bias_') == True:
            bias_list.append(i)
    
    # BMake an empty list to hold the calculated values, to be used to build the difference dataframe
    diff_squared_list = []
    
    # Iterate through the index of tubes in the master dataframe to select a reference tube
    for each_ref_tube in df.index:
        
        # Get the next reference tube data, as a series
        ref_tube = df.iloc[each_ref_tube]
        ref_tube_ID = ref_tube.tube_ID

        # Iterate through the index of tubes in the master dataframe to select a match tube
        for each_match_tube in df.index:
            
            # Get the next tube data to be compared, as a series
            match_tube = df.iloc[each_match_tube]
            match_tube_ID = match_tube.tube_ID
            
            # Dictionary container to hold the difference variables for each tube
            temp_dict = {}
            
            # Compute the squares of the differences in each current measurement
            error_sum = 0
            for bias in enumerate(bias_list):
                mismatch = ((float(match_tube.loc[bias[1]]) - float(ref_tube.loc[bias[1]]))**2)
                temp_dict[bias[1]] = mismatch
                error_sum = error_sum + mismatch

            # Add other relevant key/value pairs to the dictionary  
            temp_dict['ref_tube_ID'] = ref_tube_ID
            temp_dict['match_tube_ID'] = match_tube_ID
            temp_dict['error_sum'] = error_sum

            # Append the tube's calculated values to the diff_squared_list
            diff_squared_list.append(temp_dict)
    
    # Make an ordered list for the columns of the new difference dataframe
    new_cols_to_add = ['ref_tube_ID', 'match_tube_ID', 'error_sum']
    df_dif_cols = new_cols_to_add + bias_list
    df_dif = pd.DataFrame(diff_squared_list, columns = df_dif_cols)
    return df_dif
    
    
df_dif = build_difference_df(df, df_stats)
print("len(df_dif) = ", len(df_dif))
df_dif.head()

len(df_dif) =  27889


,ref_tube_ID,match_tube_ID,error_sum,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,10,514.934156,213.627456,92.756161,52.085089,41.165056,33.593616,25.918281,19.395216,13.601344,9.180900,5.870929,3.694084,2.402500,1.643524
2,1,100,1116.246220,518.563984,226.924096,121.484484,81.993025,58.018689,40.043584,27.133681,17.783089,11.135569,6.630625,3.625216,1.896129,1.014049
3,1,102,170.397792,51.437584,29.997529,24.651225,20.839225,16.345849,11.478544,7.546009,4.401604,2.262016,1.004004,0.346921,0.084681,0.002601
4,1,103,1805.026199,878.470321,397.364356,194.463025,126.315121,81.812025,51.108201,31.674384,19.158129,11.390625,6.620329,3.667225,1.968409,1.014049


# Find the best matches for a particular tube_ID

In [ ]:
def find_best_set_for_chosen_tube_ID(tube_to_match, tube_set_size):
    tube_to_match = str(tube_to_match)
    
    # Create a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
    df_tube_set = df_dif[(df_dif.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(tube_set_size)
    
    return df_tube_set

tube_to_match = 43
tube_set_size = 16
df_tube_set = find_best_set_for_chosen_tube_ID(tube_to_match, tube_set_size)
df_tube_set.head(tube_set_size)


In [ ]:
tube_set_score = df_tube_set.error_sum.sum()
tube_set_score


# STEP 5: Find the best matches for any tube set size
# Build a dataframe of the best matched sets in order

In [12]:
# Make a local version of df_dif that can be butchered
df_dif_copy = df_dif.copy()
print("Pre:", len(df_dif_copy))


def ask_tube_set_size():
    
    # Ask the user to enter the set size
    tube_set_size = input("Please enter the number of tubes per matched set:")
    
    # Build a list of tube names for the tube set size
    tube_sets_columns = ['Tube_' + str(1 + i) for i in range(int(tube_set_size))]    
    
    return tube_set_size, tube_sets_columns 


def find_best_set_for_chosen_tube_ID(df_dif_copy, tube_to_match, tube_set_size, tube_sets_columns):
    tube_to_match = str(tube_to_match)
        
    # Create a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
    df_tube_set = df_dif_copy[(df_dif_copy.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(int(tube_set_size))
        
    # Create the dictionary of the tubes in the set and the total score of the set
    tube_set_list = list(df_tube_set.match_tube_ID)
    tube_set_dict = dict(zip(tube_sets_columns, tube_set_list))   
    tube_set_score = df_tube_set.error_sum.sum()
    tube_set_dict.update({'tube_set_score':tube_set_score})

    return tube_set_dict


def find_best_set(df_dif_copy, tube_set_size, tube_sets_columns):
    
    # Make an empty list to hold the tube sets and their scores
    tube_sets_and_scores_list = []
    temp_dict = {}
    
    if 'tube_set_score' in tube_sets_columns:
        tube_sets_columns.remove('tube_set_score')

    for i in df_dif_copy.ref_tube_ID.unique():
        temp_dict = find_best_set_for_chosen_tube_ID(df_dif_copy, i, tube_set_size, tube_sets_columns)
        tube_sets_and_scores_list.append(temp_dict)

    # Add 'tube_set_score' column name at the front of the list
    if 'tube_set_score' not in tube_sets_columns: 
        tube_sets_columns.insert(0, 'tube_set_score')    
    
    # Create a dataframe for the tube sets called df_tube_sets 
    df_tube_sets = pd.DataFrame(tube_sets_and_scores_list, columns=tube_sets_columns)
    
    # Sort the tube_set_score column, take the top row (with the lowest score) as a new df 
    df_best_set_temp = df_tube_sets.sort_values('tube_set_score').head(1)
        
    return df_best_set_temp
   

   
    
# Make an empty dataframe to hold all the sets and their scores
df_best_tube_sets = pd.DataFrame()

# Get the tube_set_size and the tube_sets_columns from the ask_tube_set_size function
tube_set_size, tube_sets_columns = ask_tube_set_size()

# Get the best tube set from the batch
df_best_set_temp = find_best_set(df_dif_copy, tube_set_size, tube_sets_columns) 

# Append the best tube set to the df_best_tube_sets dataframe
df_best_tube_sets = df_best_tube_sets.append(df_best_set_temp)

# Remove the best tube set from df_dif_copy
tube_delete_list = list(df_best_set_temp.iloc[0])
tube_delete_list.pop(0)
print(tube_delete_list)
df_dif_copy = df_dif_copy[~df_dif_copy.ref_tube_ID.isin(tube_delete_list)]
df_dif_copy = df_dif_copy[~df_dif_copy.match_tube_ID.isin(tube_delete_list)]

set_count = 1
print("Tube set #", set_count, "has been found.")




while len(df_dif_copy.ref_tube_ID.unique()) >= int(tube_set_size):
    print(len(df_dif_copy))
    # Get all the rest of the sets from the batch
    df_best_set_temp = find_best_set(df_dif_copy, tube_set_size, tube_sets_columns)
    
    # Append the best tube set to the df_best_tube_sets dataframe
    df_best_tube_sets = df_best_tube_sets.append(df_best_set_temp)

    # Remove the best tube set from df_dif_copy
    tube_delete_list = list(df_best_set_temp.iloc[0])
    tube_delete_list.pop(0)
    print(tube_delete_list)
    df_dif_copy = df_dif_copy[~df_dif_copy.ref_tube_ID.isin(tube_delete_list)]
    df_dif_copy = df_dif_copy[~df_dif_copy.match_tube_ID.isin(tube_delete_list)]

    set_count += 1
    print("Tube set #", set_count, "has been made, with a score of:", str(df_best_set_temp.tube_set_score.iloc[0]))


Pre: 27889
Please enter the number of tubes per matched set:16
['117', '194', '123', '15', '85', '135', '17', '5', '32', '82', '138', '148', '79', '13', '14', '54']
Tube set # 1 has been found.
22801
['84', '89', '118', '122', '70', '26', '66', '125', '63', '53', '140', '95', '18', '44', '106', '3']
Tube set # 2 has been made, with a score of: 54.124114
18225
['104', '109', '143', '192', '149', '196', '146', '64', '132', '102', '33', '142', '128', '94', '88', '136']
Tube set # 3 has been made, with a score of: 68.643707
14161
['137', '99', '126', '144', '115', '72', '10', '47', '190', '68', '120', '19', '30', '49', '111', '119']
Tube set # 4 has been made, with a score of: 84.165501
10609
['98', '100', '150', '92', '86', '141', '71', '48', '52', '77', '112', '50', '96', '145', '36', '25']
Tube set # 5 has been made, with a score of: 150.968354
7569
['184', '161', '165', '178', '166', '174', '154', '180', '167', '37', '39', '6', '170', '182', '75', '20']
Tube set # 6 has been made, with

In [13]:
df_best_tube_sets

,tube_set_score,Tube_1,Tube_2,Tube_3,Tube_4,Tube_5,Tube_6,Tube_7,Tube_8,Tube_9,Tube_10,Tube_11,Tube_12,Tube_13,Tube_14,Tube_15,Tube_16
16,49.577323,117,194,123,15,85,135,17,5,32,82,138,148,79,13,14,54
137,54.124114,84,89,118,122,70,26,66,125,63,53,140,95,18,44,106,3
5,68.643707,104,109,143,192,149,196,146,64,132,102,33,142,128,94,88,136
20,84.165501,137,99,126,144,115,72,10,47,190,68,120,19,30,49,111,119
102,150.968354,98,100,150,92,86,141,71,48,52,77,112,50,96,145,36,25
41,202.521719,184,161,165,178,166,174,154,180,167,37,39,6,170,182,75,20
26,239.799158,175,181,162,176,173,168,156,151,179,160,158,172,186,187,164,56
20,371.915362,2,195,189,9,113,51,43,45,58,28,35,38,139,97,31,107
38,419.492066,90,65,81,116,34,67,76,124,80,103,108,29,22,16,87,130
17,3875.261342,60,7,8,62,155,1,42,177,153,24,12,46,147,133,4,21


In [14]:
len(df_best_tube_sets)

10

In [ ]:
len(df_dif_copy)


In [ ]:
# Make a list of the tubes to be deleted from df_dif_copy
tube_delete_list = list(df_best_tube_sets.iloc[0])
tube_delete_list.pop(0)

# Delete the tubes in the set from df_dif_copy since they're no longer available.
df_dif_copy = df_dif_copy[~df_dif_copy.ref_tube_ID.isin(tube_delete_list)]

# Check to see if the first value of the list is in the result
tube_delete_list[0] in df_dif_copy2.ref_tube_ID

len(df_dif_copy)

In [ ]:
len(df_dif_copy)

In [ ]:
# Make a new dataframe excluding all references to a certain tube
df_dif_copy_deletedstuff = df_dif_copy[(df_dif_copy.ref_tube_ID != '1')].sort_values('error_sum')

In [ ]:
len(df_dif_copy_deletedstuff)

In [ ]:
df_dif_copy_deletedstuff = df_dif_copy_deletedstuff[(df_dif_copy_deletedstuff.ref_tube_ID != df_dif_copy_deletedstuff.match_tube_ID)]

In [ ]:
len(df_dif_copy_deletedstuff)

In [ ]:
# Is it possible to pass a list of tubes to drop using the filter?
df_dif_copy_xyz = df_dif_copy_deletedstuff[(df_dif_copy_deletedstuff.ref_tube_ID == '2')]
delete_list = df_dif_copy_xyz.ref_tube_ID
# df_dif_copy_xyz = df_dif_copy_xyz.drop(delete_list)
df_dif_copy_xyz 

In [ ]:


df_dif_copy2 = df_dif_copy.copy()
# for tube in tubes_to_be_removed:
#     print(tube)

# thing = df_dif_copy2[(df_dif_copy2.ref_tube_ID != df_best_set.loc[0, 'Tube_1'])]

# Miscellaneous exeriments with ipywidgets

In [ ]:
import ipywidgets as wg
from ipywidgets import Layout, Box
from IPython.display import display


# Make a list of all the tube_ID names, then build a select box with those values
tube_list = list(df.loc[:, 'tube_ID'])
tube_list.sort(key=int)
tube_ID_select_box = wg.Select(options=tube_list, value=tube_list[0], description='Reference Tube', layout=Layout(width='150px'), disabled=False)

# Make a list of the tubes for the "Matched Tubes" selectMultiple box
matched_list = ['1', '33', '67', '22', '45']
matched_tube_box = wg.SelectMultiple(options=matched_list, layout=Layout(display="flex", flex_flow='column', width='350px'), description="Matched Tubes") 


items = [tube_ID_select_box, matched_tube_box]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box = Box(children=items, layout=box_layout)

box


# def respond(sender):
    # pass the chosen Tube_ID to a function that looks up the resulting match list, then fill the match list box
    
    
# display(tube_ID_select_box,matched_tube_box)
# tube_ID_select_box.observe(respond)

# Other Miscellaneous stuff

##### Apply a function to a column
df.Bias_2.max()

In [ ]:
# Applymap is useful for applying a function to all data
# For example, converting all data from floats to ints
# df.loc[2:7, 'Bias_2': 'Bias_50'].applymap(int)

In [ ]:
# List all available magics
% lsmagic

In [ ]:
# Display environmental variables
env_variables = %env
current_path = env_variables.get('PATH')
current_path

In [ ]:
mydict = % env
mydict.keys()

In [ ]:
% env

In [ ]:
# % time x = range(10000)
% timeit x = range(10000)
max(x)